In [1]:
import torch
import cv2
import random
import numpy as np
from tqdm.auto import tqdm
import os
import math
import glob
import nibabel as nib
import matplotlib.pyplot as plt

In [2]:
from concurrent.futures import ThreadPoolExecutor

# Data Preprocessing

In [3]:
train_dir = './train_set_raw'
train_dir_processed = './train_set_processed'

sample_dir = glob.glob(os.path.join(train_dir, "*", "*"))

In [4]:
'''
for sample in tqdm(sample_dir):
    label_img = np.asanyarray(nib.load(os.path.join(sample, os.path.split(sample)[-1] + '_seg.nii')).dataobj)
    flair_image = np.asanyarray(nib.load(os.path.join(sample, os.path.split(sample)[-1] + '_flair.nii')).dataobj)
    t1_image = np.asanyarray(nib.load(os.path.join(sample, os.path.split(sample)[-1] + '_t1.nii')).dataobj)
    t1ce_image = np.asanyarray(nib.load(os.path.join(sample, os.path.split(sample)[-1] + '_t1ce.nii')).dataobj)
    t2_image = np.asanyarray(nib.load(os.path.join(sample, os.path.split(sample)[-1] + '_t2.nii')).dataobj)
    x_concatenated = np.stack([flair_image, t1_image, t1ce_image, t2_image], axis=-1)
    x_transposed = np.transpose(x_concatenated, (2, 3, 0, 1))
    x_reshaped = x_transposed.reshape((flair_image.shape[-1], 4, flair_image.shape[0], flair_image.shape[1])) #layer*channel*h*w
    y_transposed = np.transpose(label_img, (2,0,1)) #layer*h*w
    output_dir = os.path.join(train_dir_processed, os.path.split(sample)[-1]+'.npz')
    np.savez_compressed(output_dir, x=x_reshaped, y=y_transposed)
'''

"\nfor sample in tqdm(sample_dir):\n    label_img = np.asanyarray(nib.load(os.path.join(sample, os.path.split(sample)[-1] + '_seg.nii')).dataobj)\n    flair_image = np.asanyarray(nib.load(os.path.join(sample, os.path.split(sample)[-1] + '_flair.nii')).dataobj)\n    t1_image = np.asanyarray(nib.load(os.path.join(sample, os.path.split(sample)[-1] + '_t1.nii')).dataobj)\n    t1ce_image = np.asanyarray(nib.load(os.path.join(sample, os.path.split(sample)[-1] + '_t1ce.nii')).dataobj)\n    t2_image = np.asanyarray(nib.load(os.path.join(sample, os.path.split(sample)[-1] + '_t2.nii')).dataobj)\n    x_concatenated = np.stack([flair_image, t1_image, t1ce_image, t2_image], axis=-1)\n    x_transposed = np.transpose(x_concatenated, (2, 3, 0, 1))\n    x_reshaped = x_transposed.reshape((flair_image.shape[-1], 4, flair_image.shape[0], flair_image.shape[1])) #layer*channel*h*w\n    y_transposed = np.transpose(label_img, (2,0,1)) #layer*h*w\n    output_dir = os.path.join(train_dir_processed, os.path.spli

In [5]:
def read_nii(path):
    image = nib.load(path)
    image = np.asanyarray(image.dataobj)
    return image

In [6]:
def process_sample(sample):
    # Fetch the images
    label_img = read_nii(os.path.join(sample, os.path.split(sample)[-1] + '_seg.nii'))
    flair_image = read_nii(os.path.join(sample, os.path.split(sample)[-1] + '_flair.nii'))
    t1_image = read_nii(os.path.join(sample, os.path.split(sample)[-1] + '_t1.nii'))
    t1ce_image = read_nii(os.path.join(sample, os.path.split(sample)[-1] + '_t1ce.nii'))
    t2_image = read_nii(os.path.join(sample, os.path.split(sample)[-1] + '_t2.nii'))
    
    # Process the raw images, denoted as x
    x_concatenated = np.stack([flair_image, t1_image, t1ce_image, t2_image], axis=-1)
    x_transposed = np.transpose(x_concatenated, (2, 3, 0, 1))
    x_reshaped = x_transposed.reshape(flair_image.shape[-1], 4, flair_image.shape[0], flair_image.shape[1]) #layer*channel*h*w
    
    # Process the label, denoted as y
    y_transposed = np.transpose(label_img, (2,0,1)) #layer*h*w
    
    # Save the two in a single npz file
    output_dir = os.path.join(train_dir_processed, os.path.split(sample)[-1]+'.npz')
    np.savez_compressed(output_dir, x=x_reshaped, y=y_transposed)

In [7]:
# Multiprocess on preprocessing the dataset
if not os.listdir(train_dir_processed):
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        list(tqdm(executor.map(process_sample, sample_dir), total=len(sample_dir)))
else:
    print('Found the preprocessed dataset folder not empty. Data preprocessing skipped.')

Found the preprocessed dataset folder not empty. Data preprocessing skipped.
